In [61]:
import tensorflow as tf
import numpy as np

In [137]:
english = [['I','am','a','boy'], ['I','am', 'a', 'girl'], ['You', 'are', 'so', 'beautiful']]
korean = [['나','는','소년','이다'], ['나','는','소녀','다'], ['당신','은','너무','아름답다']]

In [138]:
seq_data = np.stack((english, korean), 1)

In [64]:
char_arr = []
for seq in seq_data:
    char_arr += seq
char_arr += ['<S>', '</S>', '<P>']
char2num = {c:i for i, c in enumerate(set(char_arr))}
num2char = {i:c for i, c in enumerate(char2num.keys())}
dic_len = len(char2num)

In [65]:
char2num

{'</S>': 17,
 '<P>': 3,
 '<S>': 9,
 'I': 14,
 'You': 4,
 'a': 7,
 'am': 18,
 'are': 21,
 'beautiful': 6,
 'boy': 12,
 'girl': 13,
 'so': 11,
 '나': 15,
 '너무': 0,
 '는': 16,
 '다': 19,
 '당신': 1,
 '소녀': 20,
 '소년': 2,
 '아름답다': 10,
 '은': 5,
 '이다': 8}

In [108]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    for i, seq in enumerate(seq_data):
        input = []
        output = []
        target = []
        
        for token in seq[0]:
            input.append(char2num[token])
        input_batch.append(np.eye(dic_len)[input])
        output.append(char2num['<S>'])
        for token in seq[1]:
            output.append(char2num[token])
            target.append(char2num[token])
        target.append(char2num['</S>'])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)

    return input_batch, output_batch, target_batch       

In [109]:
input_batch, output_batch, target_batch = make_batch(seq_data)

In [113]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 500
n_class = n_input = dic_len

In [114]:
tf.reset_default_graph()
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

In [115]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

In [116]:
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state = enc_states, dtype=tf.float32)

In [117]:
logits = tf.layers.dense(outputs, n_class, activation=None)

In [118]:
weights = tf.ones(shape=[3,5], dtype=tf.float32)
cost = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(logits=logits, targets=targets, weights=weights))

In [119]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [120]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    if epoch % 50 == 49:
        print('Epoch:', '%03d' % (epoch+1), 'cost =', '{:.6f}'.format(loss))
    
print('optimization finished!')

Epoch: 050 cost = 0.059496
Epoch: 100 cost = 0.006288
Epoch: 150 cost = 0.001487
Epoch: 200 cost = 0.000703
Epoch: 250 cost = 0.002068
Epoch: 300 cost = 0.000539
Epoch: 350 cost = 0.001674
Epoch: 400 cost = 0.000639
Epoch: 450 cost = 0.000760
Epoch: 500 cost = 0.000189
optimization finished!


In [148]:
def translate(sentence):
    seq_data = [[sentence.split(' '), len(sentence.split(' '))*['<P>']]]
    input_batch, output_batch, target_batch = make_batch(seq_data)
    prediction = tf.argmax(logits, 2)
    result = sess.run(prediction, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    decoded = [num2char[i] for i in result[0]]
    
    end = decoded.index('</S>')
    translated = ' '.join(decoded[:end])
    
    return translated

In [149]:
seq_data = [['I am a boy'.split(' '), len('I am a boy'.split(' '))*['<P>']]]

In [150]:
seq_data

[[['I', 'am', 'a', 'boy'], ['<P>', '<P>', '<P>', '<P>']]]

In [151]:
print('===translation test===')
print('I am a boy ->', translate('I am a boy'))
print('I am a girl ->', translate('I am a girl'))
print('You are so beautiful ->', translate('You are so beautiful'))
print('I am so beautiful ->', translate('I am so beautiful'))

===translation test===
I am a boy -> 나 는 소년 이다
I am a girl -> 나 는 소녀 다
You are so beautiful -> 당신 은 너무 아름답다
I am so beautiful -> 나 는 소년 이다
